# Detailed Report

In [ ]:
import urlparse
url = urlparse.urlsplit(jupyter_notebook_url)
pk = urlparse.parse_qs(url.query)['pk'][0] # WorkCalculation

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
%matplotlib inline
from aiida.orm.data.remote import RemoteData
from aiida.orm import load_node
import nanoribbon

import matplotlib.pyplot as plt
import matplotlib.image as image
from glob import glob
from os import path
import re
import numpy as np
import gzip

import ipywidgets as ipw
from IPython.display import display, clear_output

In [ ]:
def read_cube(fn):
    lines = gzip.open(fn).readlines()
    header = np.fromstring("".join(lines[2:6]), sep=' ').reshape(4,4)
    natoms, nx, ny, nz = header[:,0].astype(int)
    cube = np.fromstring("".join(lines[natoms+6:]), sep=' ').reshape(nx, ny, nz)
    return(cube)

## Kohn Sham Orbitals

In [ ]:
node = load_node(pk=int(pk))
all_orbitals = [fn for fn in node.out.hartree.get_folder_list() if fn.endswith("_orbital.cube.gz")]

def on_orb_change(c):
    global cube
    fn = node.out.hartree.get_abs_path(orb_select.value)
    cube = read_cube(fn)
    height_slider.max=cube.shape[2]-1
    on_height_change(None)

orb_select = ipw.Dropdown(options=all_orbitals, description="Orbitals")
orb_select.observe(on_orb_change, names='value')
display(orb_select)

In [ ]:
def on_height_change(c):
    with plt_out:
        clear_output()
        z = height_slider.value
        plt.gcf().dpi=100.0
        plt.imshow(cube[:,:,z])
        plt.colorbar()
        plt.show()

height_slider = ipw.IntSlider(description="Height", min=0, max=1, continuous_update=False)   
height_slider.observe(on_height_change, names='value')
plt_out = ipw.Output()
display(height_slider, plt_out)

on_orb_change(None)

In [ ]:
# node = load_node(pk=int(pk))
# bands_calc_obj, scf_calc_obj, hartree_calc_obj, struct = nanoribbon.get_calc_obj(node)
# remote_datas = [i for i in bands_calc_obj.get_outputs() if isinstance(i, RemoteData)]
# assert(len(remote_datas)==1)
# remote_data = remote_datas[0]

In [ ]:
#fig, ax = plt.subplots(figsize=(5,5))
#nanoribbon.plot_thumbnail(ax, struct.get_ase())

In [ ]:
# def build_img_view(remote_data, label, description):
#     nodes = [ i for i in remote_data.get_outputs() if i.label.endswith(label)]
#     if(len(nodes)!=1):
#         print("Node for %s not found"%description)
#         return

#     png_dir = nodes[0].out.retrieved.get_abs_path()+'/path'
#     allpngs = glob(png_dir+'/*.png')

#     def on_change(c):
#         clear_output()
#         if "Please select" in dropdown.value:
#             return
#         s = 5 if checkbox.value else 10
#         fig, ax = plt.subplots(figsize=(s,s))
#         ax.xaxis.set_visible(False)
#         ax.yaxis.set_visible(False)
#         fn = png_dir + "/" + dropdown.value
#         img = image.imread(fn)
#         ax.imshow(img)

#     options = ["Please select"] + [path.basename(fn) for fn in allpngs]
#     dropdown = widgets.Dropdown(options=options, description=description)
#     checkbox = widgets.Checkbox(value=True, description='thumbnail')

#     dropdown.observe(on_change, names='value')
#     checkbox.observe(on_change, names='value')
#     display(widgets.HBox([dropdown,checkbox]))

In [ ]:
#build_img_view(remote_data, " KS", "KS Figure")

In [ ]:
#build_img_view(remote_data, " STS FILLED", "STS Filled")

In [ ]:
#build_img_view(remote_data, " STS EMPTY", "STS Empty")